#  디스크립터란

- 클래스의 속성을 보호하기 위해 만드는 특별한 클래스이다.
- 디스크립터로 만드는 속성을 보호 속성으로 만든다.
- 속성에 대한 접근을 이름으로 조회하고 갱신할 수 있다.

- 기본 프로토콜 규약에서는 3가지가 있다.

> `__get__`, `__set__`, `__delete__` 중 하나가 구현되어 있는 클래스를 디스크립터 클래스

## 1.  디스크립터를 만들고 내부 인스턴스에 속성 넣기

## 디스크립터 클래스는 정의

- 디스크립터 객체를 생성할 때 값을 관리하는 필드와 이 속성을 처리하는 3개의 메소드를 정의한다.
- 값 저장은 일단 디스크립터 객체에 만들어서 보관하는 것부터 알아본다.


In [1]:
class Descriptor:
    def __init__(self, value):
        self.value = value
        
    def __get__(self, instance, owner):
        print("get")
        return self.value       # 디스크립터 객체의 값을 조회
    
    def __set__(self, instance, value):
        print("set")
        self.value = value      # 디스크립터 객체의 값을 갱신
        
    def __delete__(self, instance):
        print("delete")
        del self.value

## 반드시 클래스 속성으로 정의 

- 디스크립터 객체는 반드시 클래스 속성으로 정의한다. 


In [2]:
class A:
    value = Descriptor(10) # instance
    value1 = Descriptor(10) # instance

### 객체를 생성한다.


In [3]:
a = A()

### 디스크립터 객체에 값을 보관해서 클래스 A의 객체에는 아무런 속성도 없다.

In [4]:
a.__dict__

{}

### 값을 조회하기

- 디스크립터는 클래스 내부의 속성을 확인하고 이를 이용해서 내부의 메소드를 호출해서 처리
- 클래스 A의 네임스페이스를 조회하면 두 개의 디스크립터 속성이 만들어져 있다.

In [5]:
A.__dict__

mappingproxy({'__module__': '__main__',
              'value': <__main__.Descriptor at 0x7fa5792dae10>,
              'value1': <__main__.Descriptor at 0x7fa5792da490>,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

### 속성의 값을 조회하기

In [6]:
type(a).__dict__['value']

In [7]:
type(a).__dict__['value'].__get__

<bound method Descriptor.__get__ of <__main__.Descriptor object at 0x7fa5792dae10>>

In [8]:
type(a).__dict__['value'].__get__(a, type(a))

get


10

In [9]:
a.value     # 디스크립터 클래스의 __get__ 메소드를 조회한다. 

get


10

###  디스크립터는 클래스의 속성을 읽어서 갱신도 함

In [10]:
type(a).__dict__['value'].__set__(a, 12)

set


In [11]:
a.value

get


12

In [12]:
a.value1 = 13  

set


In [13]:
a.value1

get


13

###  실제 삭제된 거을 디스크리터로 만들어진 객체 

In [14]:
type(a).__dict__['value'].__delete__(a)

delete


In [15]:
type(a).__dict__

mappingproxy({'__module__': '__main__',
              'value': <__main__.Descriptor at 0x7fa5792dae10>,
              'value1': <__main__.Descriptor at 0x7fa5792da490>,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

In [16]:
try : 
    a.value
except Exception as e :
    print(e)

get
'Descriptor' object has no attribute 'value'


In [17]:
del a.value1 

delete


In [18]:
type(a).__dict__

mappingproxy({'__module__': '__main__',
              'value': <__main__.Descriptor at 0x7fa5792dae10>,
              'value1': <__main__.Descriptor at 0x7fa5792da490>,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

In [19]:
try : 
    a.value1
except Exception as e :
    print(e)

get
'Descriptor' object has no attribute 'value'


## 다시 세팅하면 디스크립터가 작동되어 처리됨 

In [20]:
a.value =123

set


In [21]:
a.value

get


123

In [22]:
type(a).__dict__['value1'].__set__(a,999)

set


In [23]:
a.value1

get


999

# 2.  디스크립터 프로토콜이 없다면..

- 속성을 관리하는 클래스를 정의하고 메인 클래스의 속성에 객체를 생성한다.
- 메인 클래스 속성 즉 객체에 속성을 추가한다.
- 이 속성에 정보를 조회한다.
- 점연산자를 연속적으로 사용해야 한다.
- 디스크립터 클래스를 만들고 처리하는 것보다 더 복잡한 구조를 가진다.

### 일반 클래스를 정의한다. 

In [24]:
class D:
    def __getattribute__(self, name):
        print("getattribute")
        return super().__getattribute__(name)


### 클래스 내부의 속성에 객체를 할당한다.

In [25]:
class A:
    value = D()

### 객체 내부의 value 속성은 객체이미므로 이 객체의 속성 data를 추가한다.

In [26]:
a = A()
a.value.data = 1

### 객체 내부의 data 값을 조회한다. 

In [27]:
a.value.data

getattribute


1

## 3.  읽기 전용  `__get__`  만 가진 디스크립터

- 메인 클래스의 속성을 읽기 전용으로만 만들 수 있다.

In [28]:
class FirstDescriptor:
    def __init__(self, value):
        self.value = value
        
    def __get__(self, instance, owner):
        print("get")
        return self.value
    

class A:
    a = FirstDescriptor(3)

In [29]:
a = A()
a.a

get


3

### 디스크립터 읽기 전용 만들기

- `__set__` 을 정의했지만 예외를 발생시켜서 읽기 전용으로 만들 수 있다.  



In [30]:
class Descriptor:
    def __init__(self, value):
        self.value = value
        
    def __get__(self, instance, onwer):
        print("get")
        return self.value
    
#     def __set__(self, instance, value):
#         print("set")
#         self.value = value  # 쓰기 가능
        
    def __set__(self, instance, value):
        # 읽기전용
        print("set")
        raise AttributeError("read only")
        


In [31]:
class A:
    a = Descriptor(3)

In [32]:
a = A()

In [33]:
a.a

get


3

In [34]:
try : 
    a.a = 12
except Exception as e :
    print(e)

set
read only


# 4. 디스크립터에서 메인 클래스의 객체로 저장하는 위치

- 실제 처리할 때는 메인클래스의 객체 내부의 속성으로 값을 처리한다.
- 메인 클래스의 인스턴스를 받아서 저장할 위치를 인스턴스로 변경한다.

### 디스크립터에 메인 클래스의 객체에 저장 처리를 하지 않았다

In [35]:
class descriptor:
    def __get__(self, instance, owner):
        print('get')
        return instance.value
    
    def __set__(self, instance, value):
        print('set')
        instance.value = value


In [36]:
### 메인클래스 속성에 디스크립터를 할당하지만 실제 객체와 동일한 변수가 없다.

In [37]:
class A:
    x = descriptor()

In [38]:
a = A()

In [39]:
a.__dict__

{}

In [40]:
a.x = 123

set


In [41]:
a.__dict__

{'value': 123}

In [42]:
try :
    a.x
except Exception as e :
    print(e)

get


### 인스턴스 객체 내에 속성 저장하고 처리하기

In [43]:
class descriptor1:
    
    def __init__(self, name):
        self.name = "_" + name 
        
    def __get__(self, instance, owner):
        print('get', instance)
        return instance.__dict__.get(self.name,0) 
    
    def __set__(self, instance, value):
        print('set')
        instance.__dict__[self.name] = value



In [44]:
class AA:
    x = descriptor1('x')

In [45]:
aa = AA()

In [46]:
aa.x

get <__main__.AA object at 0x7fa5793356d0>


0

In [47]:
aa.x = 100

set


In [48]:
aa.__dict__

{'_x': 100}

In [49]:
aa.x

get <__main__.AA object at 0x7fa5793356d0>


100

### 메인 클래스 속성을 가져오기

In [50]:
class descriptor2:
    
    def __set_name__(self, owner, name ):
        self.name = "_"+ name
        
    def __get__(self, instance, owner):
        print('get', instance)
        return instance.__dict__.get(self.name,0) 
    
    def __set__(self, instance, value):
        print('set')
        instance.__dict__[self.name] = value

In [51]:
class AAA:
    x = descriptor2()

In [52]:
aaa = AAA()

In [53]:
aaa.x = 100

set


In [54]:
aaa.x

get <__main__.AAA object at 0x7fa57934be50>


100

## 5. 조회할 때 속성을 초기화하기

In [55]:
class descriptor3:
    
    def __set_name__(self, owner, name ):
        self.name = "_"+ name
        
    def __get__(self, instance, owner):
        print('get', instance)
        return instance.__dict__.setdefault(self.name,0) 
    
    def __set__(self, instance, value):
        print('set')
        instance.__dict__[self.name] = value

In [56]:
class AAAA :
    x = descriptor3()

In [57]:
aaaa = AAAA()

In [58]:
aaaa.x

get <__main__.AAAA object at 0x7fa57933c150>


0

In [59]:
aaaa.__dict__

{'_x': 0}

In [60]:
aaaa.x =100

set


In [61]:
aaaa.__dict__

{'_x': 100}

In [62]:
aaaa.x

get <__main__.AAAA object at 0x7fa57933c150>


100

## 6.  함수를 저장해서 처리하는 디스크립터 만들기 

- 디스크립터 클래스를 만들고 데코레이터를 처리할 수 있다.
- 이름으로 접근하면 내부으 함수가 실행된다.


### 디스크립터 조회할 때 저장된 함수를 반환한다

In [63]:
class descriptor:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        print("get")
        return self.func(instance) # 메소드는 첫번째 인자로 self 자기자신을 받아야되기 때문에

### 디스크립터로 메소드를 데코레이터 처리한다.

In [64]:
class A:
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c
    
    @descriptor
    def func(self):
        return self.a + self.b + self.c


In [65]:
a = A(1, 2, 3)

In [66]:
a.func # __call__ X

get


6

### 함수의 인자를 받도록 수정

- 부분 함수 처리를 위해 pattial 로 처리한다. 
- 이름으로 조회하면 부분함수가 반환되고 메소드의 인자를 추가적으로 넣어서 처리할 수 있다

In [67]:
from functools import partial


class descriptor:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        print('get')
        return partial(self.func, instance)

In [68]:
class A:
    @descriptor
    def func(self, a, b, c):
        return a + b + c

In [69]:
a = A()

In [70]:
a.func(1, 2, 3)

get


6

## 7. 도전 : 디스크립터로 구현

### 인스턴스를 생성할때 반지름을 입력받는 원 객체 / 원의 속성을 가지고 있는..

> 반지름은 mutable    
> 그 외에 지름, 둘레, 넓이는 immutable

### 함수를 저장하는 방식으로

### 갱신가능한 디스크립터 정의

In [71]:
class MutableAttribute:
    def __init__(self, value=None):
        self.value = value
        
    def __get__(self, instance, owner):
        return self.value
    
    def __set__(self, instance, value):
        self.value = value
        
    def __delete__(self, instance):
        del self.value



### 갱신 불가능한 디스크립터 정의

In [72]:
class ImmutableAttribute:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        return self.func(instance)
    
    def __set__(self, instance, value):
        raise AttributeError("read only")
        
    def __delete__(self, instance):
        raise AttributeError("read only")

### 메인 클래스 정의하고 메소드를 변경불가능하도록 처리

In [73]:
class Circle:
    pi = 3.1415
    radius = MutableAttribute(10)
    diameter = ImmutableAttribute(lambda self : self.radius * 2)
    
#     @ImmutableAttribute
#     def diameter(self):
#         return self.radius * 2
    
    @ImmutableAttribute
    def circumference(self):
        return self.radius * self.pi * 2
    
    @ImmutableAttribute
    def area(self):
        return self.radius **2 * 2

In [74]:
c = Circle()

In [75]:
c.radius

10

In [76]:
c.diameter

20

In [77]:
c.area

200

In [78]:
c.radius = 100

In [79]:
c.area

20000

In [80]:
try :
    c.area = 123
except Exception as e :
    print(e)

read only
